In [3]:
import urllib.request
import json
import re
import datetime
from sys import intern
import pytz

# def fetch_date(page, date, entity):
def fetch_date(page, date):
    date_str = datetime.datetime.fromisoformat(date).strftime("%d-%m-%Y")
    url = f"https://www.nordpoolgroup.com/api/marketdata/page/{page}?currency=,EUR,EUR,EUR&endDate={date_str}"
    response = urllib.request.urlopen(url)
    data = json.load(response)
    return data

def extract_rows(data):
    rows = []
    for row in data["data"]["Rows"]:
        if row["IsExtraRow"] or row["Parent"] is not None:
            continue
        row_dict = {}
        for column in row["Columns"]:
            if column["Name"] == "":
                continue
            row_dict[column["Name"]] = column["Value"]
        if len(row_dict) > 0:
            rows.append(row_dict)    
    return rows

def extract_dates(product_str):
    tz = pytz.timezone("Europe/Copenhagen")
    date_match = re.match(r"PH-(\d{8})-(\d{2})(X?)", product_str)
    date_str, hour_str, dst_flag = date_match.groups()
    is_dst = (dst_flag != "X")
    date = datetime.datetime.strptime(date_str, "%Y%m%d")
    start_time = datetime.time(hour=int(hour_str)-1)
    dt = datetime.datetime.combine(date, start_time)
    start_loc = tz.localize(dt, is_dst=is_dst) # is_dst is ignored unless there is ambiguity
    end_loc = (start_loc.astimezone(datetime.timezone.utc) + datetime.timedelta(hours=1)).astimezone(tz)
    return start_loc.isoformat(), end_loc.isoformat()

numerical_columns = ["High", "Low", "Last", "Avg", "Volume"]

def format_row(row):
    record = {}
    dt_start, dt_end = extract_dates(row["Product"])
    record["datetime_start"] = dt_start
    record["datetime_end"] = dt_end
    for col, val in row.items():
        if col in numerical_columns:
            val = float(val.replace(",", "."))
        record[intern(col)] = val
    return record

class NordpoolSpot:
    def __init__(self, descriptor):
        self.descriptor = descriptor
        if descriptor.get("dataset") != "intraday":
            raise ValueError('descriptor["dataset"] needs to have value "intraday"')
        step = datetime.timedelta(days=1)
        start_date = datetime.datetime.fromisoformat(descriptor["start_datetime"])
        end_date = datetime.datetime.fromisoformat(descriptor["end_datetime"])
        dates = [start_date + i*step for i in range(int((end_date - start_date)/step)+1)]
        self.request_dates = [date.isoformat() for date in dates]
        self.zone = descriptor["zone"]
        
    def fetch_metadata(self):
        return {
            "fields": ["datetime_start", "datetime_end", "Product"] + numerical_columns
        }
    
    def fetch_data(self):
        for request_date in self.request_dates:
            data = fetch_date(request_date, self.zone)
            rows = extract_rows(data)
            for row in rows:
                record = format_row(row)
                record["zone"] = self.zone
                yield record

In [29]:
def extract_volume_rows(data):
    tz = pytz.timezone("Europe/Copenhagen")
    rows = []
    for row in data["data"]["Rows"]:
        if row["IsExtraRow"] or row["Parent"] is not None:
            continue
        dt_start = datetime.datetime.fromisoformat(row["StartTime"])
        dt_end = datetime.datetime.fromisoformat(row["EndTime"])
        row_dict = {
            "datetime_start": tz.localize(dt_start).isoformat(), 
            "datetime_end": tz.localize(dt_end).isoformat()
        }
        for column in row["Columns"]:
            if column["Name"] == "":
                continue
            row_dict[column["Name"]] = column["Value"]
        if len(row_dict) > 0:
            rows.append(row_dict)
    return rows


In [26]:
data = fetch_date(66, "2022-12-14")

In [30]:
data

{'data': {'Rows': [{'Columns': [{'Index': 0,
      'Scale': 1,
      'SecondaryValue': None,
      'IsDominatingDirection': False,
      'IsValid': True,
      'IsAdditionalData': False,
      'Behavior': 0,
      'Name': 'Turnover at system price',
      'Value': '46 112,4',
      'GroupHeader': '',
      'DisplayNegativeValueInBlue': False,
      'CombinedName': 'Turnover at system price',
      'DateTimeForData': '0001-01-01T00:00:00',
      'DisplayName': '46 112,4_True',
      'DisplayNameOrDominatingDirection': '46 112,4',
      'IsOfficial': True,
      'UseDashDisplayStyle': False},
     {'Index': 1,
      'Scale': 1,
      'SecondaryValue': None,
      'IsDominatingDirection': False,
      'IsValid': True,
      'IsAdditionalData': False,
      'Behavior': 0,
      'Name': 'NO1 Buy',
      'Value': '5 344,8',
      'GroupHeader': '',
      'DisplayNegativeValueInBlue': False,
      'CombinedName': 'NO1 Buy',
      'DateTimeForData': '0001-01-01T00:00:00',
      'DisplayName': 

In [31]:
rows = extract_volume_rows(data)
rows

[{'datetime_start': '2022-12-14T00:00:00+01:00',
  'datetime_end': '2022-12-14T01:00:00+01:00',
  'Turnover at system price': '46 112,4',
  'NO1 Buy': '5 344,8',
  'NO1 Sell': '2 001,4',
  'NO2 Buy': '4 558,9',
  'NO2 Sell': '7 952,4',
  'NO3 Buy': '3 331,8',
  'NO3 Sell': '2 825,7',
  'NO4 Buy': '2 003,6',
  'NO4 Sell': '3 712,6',
  'NO5 Buy': '1 683,7',
  'NO5 Sell': '3 830,1',
  'SE1 Buy': '1 388,6',
  'SE1 Sell': '3 079,2',
  'SE2 Buy': '1 924,3',
  'SE2 Sell': '4 228,7',
  'SE3 Buy': '10 200,2',
  'SE3 Sell': '6 396,8',
  'SE4 Buy': '2 293,9',
  'SE4 Sell': '652,9',
  'FI Buy': '7 616,8',
  'FI Sell': '5 307,6',
  'DK1 Buy': '1 248,4',
  'DK1 Sell': '3 018,7',
  'DK2 Buy': '529,5',
  'DK2 Sell': '1 823,8',
  'EE Buy': '982,8',
  'EE Sell': '1 042,0',
  'LV Buy': '680,9',
  'LV Sell': '618,5',
  'LT Buy': '1 980,3',
  'LT Sell': '390,7'},
 {'datetime_start': '2022-12-14T01:00:00+01:00',
  'datetime_end': '2022-12-14T02:00:00+01:00',
  'Turnover at system price': '45 854,1',
  'NO1 

In [34]:
def format_volume_row(row):
    record = {k: float(v.replace(",", ".").replace(" ", "")) if not k.startswith("datetime_") else v for k, v in row.items()}
    return record

In [35]:
[format_volume_row(row) for row in rows]

[{'datetime_start': '2022-12-14T00:00:00+01:00',
  'datetime_end': '2022-12-14T01:00:00+01:00',
  'Turnover at system price': 46112.4,
  'NO1 Buy': 5344.8,
  'NO1 Sell': 2001.4,
  'NO2 Buy': 4558.9,
  'NO2 Sell': 7952.4,
  'NO3 Buy': 3331.8,
  'NO3 Sell': 2825.7,
  'NO4 Buy': 2003.6,
  'NO4 Sell': 3712.6,
  'NO5 Buy': 1683.7,
  'NO5 Sell': 3830.1,
  'SE1 Buy': 1388.6,
  'SE1 Sell': 3079.2,
  'SE2 Buy': 1924.3,
  'SE2 Sell': 4228.7,
  'SE3 Buy': 10200.2,
  'SE3 Sell': 6396.8,
  'SE4 Buy': 2293.9,
  'SE4 Sell': 652.9,
  'FI Buy': 7616.8,
  'FI Sell': 5307.6,
  'DK1 Buy': 1248.4,
  'DK1 Sell': 3018.7,
  'DK2 Buy': 529.5,
  'DK2 Sell': 1823.8,
  'EE Buy': 982.8,
  'EE Sell': 1042.0,
  'LV Buy': 680.9,
  'LV Sell': 618.5,
  'LT Buy': 1980.3,
  'LT Sell': 390.7},
 {'datetime_start': '2022-12-14T01:00:00+01:00',
  'datetime_end': '2022-12-14T02:00:00+01:00',
  'Turnover at system price': 45854.1,
  'NO1 Buy': 5190.8,
  'NO1 Sell': 1968.8,
  'NO2 Buy': 4657.2,
  'NO2 Sell': 7903.2,
  'NO3 Buy'